# 🍳 Recipe Chef Assistant with AI + MCP

Build your own culinary companion that:
1. 🔍 Researches recipes from cooking websites
2. 🧠 Remembers your dietary preferences and restrictions
3. 💡 Suggests recipes based on what you have

## How This Works

When you pass MCP tools to aisuite:
1. **aisuite handles the glue work** - converts MCP tool specs to the format your LLM needs (OpenAI, Anthropic, etc.)
2. **Automatic execution** - when the LLM requests a tool, aisuite calls the MCP server and returns results
3. **You just write natural prompts** - no need to worry about tool schemas or execution logic!

This is the power of aisuite + MCP: unified tool calling across any LLM provider.

**What you need:**
- Anthropic API key (add to `.env` file)
- Python with `uv` installed (for fetch MCP server)
- Node.js/npx installed (for memory MCP server)

**Installation:**
```bash
pip install aisuite python-dotenv
pip install 'aisuite[mcp]'  # Includes MCP client + nest_asyncio for Jupyter support
pip install uv  # For fetch MCP server
```

In [10]:
import os
import sys
from pathlib import Path

# Add parent directory to path for development
# Skip this step if you're running from an installed package
repo_root = Path().absolute().parent.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from dotenv import load_dotenv
from aisuite import Client
from aisuite.mcp import MCPClient

load_dotenv()

# Verify API key
if not os.getenv("ANTHROPIC_API_KEY"):
  raise ValueError("❌ Add ANTHROPIC_API_KEY to .env file!")

print("✓ Ready to cook up some recipes!")

✓ Ready to cook up some recipes!


## Step 1: Set Up Your Chef Assistant

We'll give the AI two tools:
- **Fetch**: Get recipes from cooking websites
- **Memory**: Remember your preferences, restrictions, and favorite recipes

In [2]:
# Set up memory file for your cooking preferences
memory_file = os.path.join(os.getcwd(), "recipe_memory.jsonl")

# Start fetch server (for getting recipes from the web)
fetch_mcp = MCPClient(
    command="uvx",
    args=["mcp-server-fetch"],
    name="fetch"
)

# Start memory server (for remembering preferences and recipes)
memory_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-memory"],
    env={"MEMORY_FILE_PATH": memory_file},
    name="memory"
)

print("🔍 Fetch server ready - can research recipes from the web")
print("🧠 Memory server ready - will remember your preferences")
print(f"📁 Recipes stored in: {memory_file}")

🔍 Fetch server ready - can research recipes from the web
🧠 Memory server ready - will remember your preferences
📁 Recipes stored in: /Users/rohit/fleet/leclerc/aisuite-prs/aisuite-main/aisuite/examples/agents/recipe_memory.jsonl


## Step 2: Set Up Your Dietary Profile

Let's tell the assistant about your dietary preferences and restrictions:

In [3]:
client = Client()

# Combine tools from both servers
all_tools = fetch_mcp.get_callable_tools() + memory_mcp.get_callable_tools()

response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{
        "role": "user",
        "content": """Store my dietary profile in memory:
        
        **Preferences:**
        - I'm vegetarian (no meat, fish, or poultry)
        - I love Italian and Thai cuisine
        - I prefer quick meals (under 30 minutes)
        - I'm intermediate skill level
        
        **Restrictions:**
        - No shellfish (allergy)
        - Lactose intolerant (use dairy alternatives when possible)
        
        **Favorites:**
        - Pasta dishes
        - Stir-fries
        - Curries
        
        Then summarize my profile back to me!"""
    }],
    tools=memory_mcp.get_callable_tools(),
    max_turns=5
)

print("="*60)
print("👤 YOUR COOKING PROFILE")
print("="*60)
print(response.choices[0].message.content)

👤 YOUR COOKING PROFILE
Perfect! I've stored your dietary profile. Here's a summary:

## Your Dietary Profile

**🥗 Diet Type:** Vegetarian (no meat, fish, or poultry)

**🍽️ Cuisine Preferences:**
- Italian cuisine
- Thai cuisine

**⚠️ Dietary Restrictions:**
- **Shellfish allergy** - must be completely avoided
- **Lactose intolerant** - dairy alternatives should be used

**❤️ Favorite Dishes:**
- Pasta dishes
- Stir-fries
- Curries

**👨‍🍳 Cooking Style:**
- Skill level: Intermediate
- Time preference: Quick meals (under 30 minutes)

Your profile is now saved, and I can use this information to provide personalized recipe suggestions and cooking advice that fits your preferences, restrictions, and lifestyle!


## Step 3: Research a Recipe & Save It

Let's find a great vegetarian pasta recipe:

In [6]:
response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{
        "role": "user",
        "content": """Research a delicious vegetarian pasta recipe from:
        - https://www.bbcgoodfood.com/recipes/collection/vegetarian-pasta-recipes
        - https://www.allrecipes.com/search?q=vegetarian+pasta
        
        Then:
        1. Make sure it fits my dietary profile (check memory!)
        2. Save the recipe name and key ingredients in memory
        3. Add notes about cook time and difficulty
        4. Give me a brief summary with why you chose it

        If you can't find any suitable recipe, let me know about your findings. Don't do more than 10 web fetches.
        
        Make it sound appetizing!"""
    }],
    tools=all_tools,
    max_turns=20
)

print("="*60)
print("🍝 RECIPE DISCOVERY")
print("="*60)
print(response.choices[0].message.content)

🍝 RECIPE DISCOVERY
Perfect! I found an amazing recipe for you! 🍝✨

## **Tomato Penne with Avocado** 

### Why I Chose This Recipe:
This is an absolute winner that hits all your requirements! It's a gorgeous Italian-Mexican fusion pasta that combines the best of both worlds - your love for Italian cuisine with exciting Mexican spices. Here's what makes it perfect:

**✅ Perfectly Fits Your Profile:**
- **100% Vegetarian** - no meat, fish, or poultry
- **Completely lactose-free** - naturally dairy-free with no cheese required!
- **No shellfish** - completely safe for your allergy
- **Quick cooking** - Ready in just 25 minutes (well under your 30-minute preference)
- **Intermediate skill level** - matches your cooking abilities perfectly

**🌟 What Makes It Special:**
This dish is absolutely stunning! Picture tender wholemeal penne tossed in a vibrant, mildly spiced tomato sauce studded with golden caramelized onions, sweet orange peppers, and bright yellow sweetcorn. The whole thing is cro

## Step 4: "What Can I Make?" - Ingredient-Based Search

Got random ingredients? Let's find what you can cook:

NOTE: We have not saved much recipes nor preferences, so you may not get an excellent suggestion! But, it will still serve the purpose of demonstrating tool usage, and can be easily extended to provide better suggestions.

In [9]:
response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{
        "role": "user",
        "content": """I have these ingredients in my kitchen:
        - Chickpeas (canned)
        - Coconut milk
        - Tomatoes
        - Onions, garlic, ginger
        - Rice
        - Various spices
        
        Based on my dietary profile and what I have:
        1. Suggest a recipe I can make
        2. Check if it matches my preferences (Thai/Italian, quick, vegetarian)
        3. Save this recipe idea to memory
        
        As the final response, give me a simple step-by-step overview of how to make it, and why you chose this for me!
        
        Be encouraging and practical!"""
    }],
    tools=all_tools,
    max_turns=10
)

print("="*60)
print("🥘 INGREDIENT-BASED RECIPE")
print("="*60)
print(response.choices[0].message.content)

🥘 INGREDIENT-BASED RECIPE
# 🍛 **Thai Chickpea Coconut Curry** - Perfect for You Tonight!

---

## ✨ **Why I Chose This Recipe:**

This is absolutely **IDEAL** for you! Here's why:
- ✅ **Thai cuisine** - one of your favorites!
- ✅ **Curry** - you specifically mentioned this as a favorite dish
- ✅ **25 minutes total** - under your 30-minute preference
- ✅ **100% vegetarian & vegan** - naturally dairy-free (no lactose issues!)
- ✅ **You have EVERY ingredient** already in your kitchen!
- ✅ **Intermediate skill level** - perfect for your abilities
- ✅ **One-pot meal** - easy cleanup!

---

## 📝 **Simple Step-by-Step Instructions:**

### **Step 1: Start Your Rice** (5 mins)
- Cook your rice according to package directions. This will be ready when your curry is done!

### **Step 2: Build Your Flavor Base** (5 mins)
- Heat a little oil in a large pan over medium heat
- Add diced onion and sauté until soft and translucent (about 5 minutes)

### **Step 3: Add Aromatics** (1-2 mins)
- Stir in min

## Cleanup

In [ ]:
fetch_mcp.close()
memory_mcp.close()
print("✓ Servers closed - your recipes are saved!")

## Recap of what we did above

Behind the scenes, aisuite did some magic:
- ✅ **Converted MCP tool schemas** to OpenAI-compatible format
- ✅ **Handled tool execution** when the LLM requested them
- ✅ **Managed async operations** for MCP server communication

MCP Tools
**Web search** - LLM could do web search due to Fetch being passed as a tool.
**Preserved your data in knowledge graph** - LLM could store your preferences and look it using server-memory being passed as a tool.

You, as an user, just wrote natural prompts - aisuite handled all the tool calling complexity - LLM returned personalized recommendations.

You built a **personal chef assistant** that:
- ✅ Researches recipes tailored to your diet
- ✅ Remembers your restrictions and preferences
- ✅ Suggests recipes based on ingredients you have
- ✅ Saves your favorite recipes and cooking notes
- ✅ Provides personalized cooking advice

**All with minimal code!**

## Try These Next

Now that you've got the basics, explore more capabilities:

### 📦 Meal Prep Planning
```python
response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{"role": "user", "content": """I want to meal prep for the week. Based on my profile:
        
        1. Suggest 2 vegetarian recipes that:
           - Reheat well
           - Stay fresh for 3-4 days
           - Match my taste (Italian/Thai)
           - Are filling and nutritious
        
        2. Save both recipes to memory as 'meal prep favorites'
        3. Give me storage tips for each"""}],
    tools=all_tools,
    max_turns=8
)
```

### 📚 Browse Your Recipe Collection
```python
response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{"role": "user", "content": """Search your memory and tell me:
        
        1. What recipes have I saved?
        2. What are my dietary restrictions and preferences?
        3. Which recipes are best for meal prep?
        4. Suggest what I should cook tonight based on my saved recipes"""}],
    tools=memory_mcp.get_callable_tools(),
    max_turns=5
)
```

### 💡 Cooking Tips & Substitutions
```python
response = client.chat.completions.create(
    model="anthropic:claude-sonnet-4-5",
    messages=[{"role": "user", "content": """I want to make a recipe that calls for:
        - Heavy cream (but I'm lactose intolerant!)
        - Parmesan cheese
        
        1. Suggest dairy-free substitutions
        2. Store these substitution tips in memory
        3. Any other dairy-free tips for Italian cooking?"""}],
    tools=all_tools,
    max_turns=6
)
```

### 🔍 Explore Your Recipe Memory
```python
# See the raw knowledge graph
import json
with open(memory_file, 'r') as f:
    for line in f.readlines()[:10]:
        entry = json.loads(line)
        if 'recipe' in entry.get('name', '').lower():
            print(f"🍽️ {entry.get('name')}")
```

### 🌟 More Fun Queries
- "Find me a healthy salad for lunch"
- "I want to try baking - suggest a beginner dessert"
- "Research authentic Thai curry recipes from https://www.bbcgoodfood.com/recipes/collection/thai-curry-recipes"
- "Plan a dinner party menu for 6 people"
- "What cooking techniques should I learn next?"
- "Create a grocery list for this week's meal prep"

## The Pattern

Setup MCP tools, and call chat.completions.create() with max_turns.

```python
# 1. Set up fetch + memory
fetch_mcp = MCPClient(command="uvx", args=["mcp-server-fetch"])
memory_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-memory"],
    env={"MEMORY_FILE_PATH": "recipe_memory.jsonl"}
)

# 2. Combine tools
tools = fetch_mcp.get_callable_tools() + memory_mcp.get_callable_tools()

# 3. Cook with AI!
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": "Find a recipe and remember it"}],
    tools=tools,
    max_turns=5
)
```

## Advanced Ideas

Extend this assistant to:
- 📊 Track nutritional goals and suggest balanced meals
- 🛒 Generate smart grocery lists with budget tracking
- 📅 Plan weekly menus automatically
- 👨‍🍳 Learn and improve from your cooking feedback
- 🌍 Explore cuisines from different cultures
- 📸 Store photos of your dishes (with filesystem MCP)
- ⏰ Set cooking timers and reminders

## Other Assistants to Build

Use the same pattern for:
- 🎬 **Movie recommendations** (see `movie_buff_assistant.ipynb`)
- ✈️ **Travel planning** that remembers your bucket list
- 📚 **Reading tracker** that knows your taste in books
- 🏋️ **Fitness coach** that tracks your progress
- 🌱 **Garden planner** that remembers what you planted

## Resources

- **Fetch Server**: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch
- **Memory Server**: https://github.com/modelcontextprotocol/servers/tree/main/src/memory
- **aisuite Documentation**: https://github.com/andrewyng/aisuite
- **More MCP Servers**: https://github.com/modelcontextprotocol/servers

**Happy cooking! Your recipes and preferences persist forever!** 🍳✨